### Parámetros del entorno (catálogo, schemas, prefijos, volumes)


In [0]:
# [00][1] Parámetros del entorno
dbutils.widgets.text("CATALOGO",                "workspace")
dbutils.widgets.text("ESQ_BRONCE",              "bronze_mb")
dbutils.widgets.text("ESQ_PLATA",               "silver_mb")
dbutils.widgets.text("ESQ_ORO",                 "gold_mb")
dbutils.widgets.text("PREFIJO_TABLA",           "mb_")

# Volumes (Unity Catalog)
dbutils.widgets.text("VOLUMEN_CATALOGO",        "workspace")
dbutils.widgets.text("VOLUMEN_ESQUEMA",         "default")
dbutils.widgets.text("VOLUMEN_LANDING",         "landing")
dbutils.widgets.text("VOLUMEN_VALIDACION",      "validation")

CATALOGO   = dbutils.widgets.get("CATALOGO").strip()
ESQ_BRONCE = dbutils.widgets.get("ESQ_BRONCE").strip()
ESQ_PLATA  = dbutils.widgets.get("ESQ_PLATA").strip()
ESQ_ORO    = dbutils.widgets.get("ESQ_ORO").strip()
PREFIJO    = dbutils.widgets.get("PREFIJO_TABLA").strip()

V_CAT  = dbutils.widgets.get("VOLUMEN_CATALOGO").strip()
V_ESQ  = dbutils.widgets.get("VOLUMEN_ESQUEMA").strip()
V_LAND = dbutils.widgets.get("VOLUMEN_LANDING").strip()
V_VAL  = dbutils.widgets.get("VOLUMEN_VALIDACION").strip()

spark.sql(f"USE CATALOG {CATALOGO}")

# Derivo rutas útiles (solo informativas)
RUTA_LANDING = f"/Volumes/{V_CAT}/{V_ESQ}/{V_LAND}"
RUTA_VALID   = f"/Volumes/{V_CAT}/{V_ESQ}/{V_VAL}"

display({
  "CATALOGO": CATALOGO,
  "ESQ_BRONCE": ESQ_BRONCE,
  "ESQ_PLATA": ESQ_PLATA,
  "ESQ_ORO": ESQ_ORO,
  "RUTA_LANDING": RUTA_LANDING,
  "RUTA_VALIDACION": RUTA_VALID
})


## Creación de schemas y selección de catálogo

In [0]:
# [00][2] Creo/aseguro los esquemas de trabajo
spark.sql(f"CREATE SCHEMA IF NOT EXISTS {CATALOGO}.{ESQ_BRONCE}")
spark.sql(f"CREATE SCHEMA IF NOT EXISTS {CATALOGO}.{ESQ_PLATA}")
spark.sql(f"CREATE SCHEMA IF NOT EXISTS {CATALOGO}.{ESQ_ORO}")
print("[INFO] Esquemas disponibles.")


### Nombres de tablas y checkpoints

In [0]:
# [00][3] Nombres de tablas canónicas (evito hardcode en los demás notebooks)
T_BRONCE   = f"{CATALOGO}.{ESQ_BRONCE}.{PREFIJO}events_bronce"
T_ESTADO   = f"{CATALOGO}.{ESQ_BRONCE}.{PREFIJO}archivos_ingeridos"
T_PLATA    = f"{CATALOGO}.{ESQ_PLATA}.{PREFIJO}events_silver"
T_QUAR     = f"{CATALOGO}.{ESQ_PLATA}.{PREFIJO}events_quarantine"
T_LOG      = f"{CATALOGO}.{ESQ_ORO}.{PREFIJO}ingestion_log"
T_RUNNING  = f"{CATALOGO}.{ESQ_ORO}.{PREFIJO}running_stats"

display({
  "T_BRONCE": T_BRONCE, "T_ESTADO": T_ESTADO,
  "T_PLATA": T_PLATA, "T_QUAR": T_QUAR,
  "T_LOG": T_LOG, "T_RUNNING": T_RUNNING
})


### DDL capa Bronce y tabla de estado

In [0]:
# [00][4] Tablas base de Bronce (raw) y Estado (idempotencia por archivo)
spark.sql(f"""
CREATE TABLE IF NOT EXISTS {T_BRONCE} (
  timestamp     STRING,
  price         STRING,
  user_id       STRING,
  _ingest_ts    TIMESTAMP,
  _source_file  STRING,
  _batch_id     STRING
) USING DELTA
""")

spark.sql(f"""
CREATE TABLE IF NOT EXISTS {T_ESTADO} (
  file_path     STRING,
  processed_at  TIMESTAMP
) USING DELTA
""")
print("[INFO] Tablas Bronce y Estado listas.")


### DDL capa Plata (tabla válida + quarantine)

In [0]:
# [00][5] Tablas de Plata: válidos tipificados y quarantine para auditoría
spark.sql(f"""
CREATE TABLE IF NOT EXISTS {T_PLATA} (
  ts            TIMESTAMP,
  price         DOUBLE,
  user_id       STRING,
  _ingest_ts    TIMESTAMP,
  _source_file  STRING,
  _batch_id     STRING
) USING DELTA
""")

spark.sql(f"""
CREATE TABLE IF NOT EXISTS {T_QUAR} (
  timestamp_raw STRING,
  price_raw     STRING,
  user_id_raw   STRING,
  reason        STRING,
  _ingest_ts    TIMESTAMP,
  _source_file  STRING,
  _batch_id     STRING
) USING DELTA
""")
print("[INFO] Tablas Plata y Quarantine listas.")


### DDL capa Oro (log y acumulado) + siembra de running_stats

In [0]:
# [00][6] Tablas de Oro: log de ingesta por lote y acumulado incremental
spark.sql(f"""
CREATE TABLE IF NOT EXISTS {T_LOG} (
  batch_id     STRING,
  rows_loaded  BIGINT,
  sum_price    DOUBLE,
  min_price    DOUBLE,
  max_price    DOUBLE,
  min_ts       TIMESTAMP,
  max_ts       TIMESTAMP,
  processed_at TIMESTAMP
) USING DELTA
""")

spark.sql(f"""
CREATE TABLE IF NOT EXISTS {T_RUNNING} (
  id         INT,
  row_count  BIGINT,
  sum_price  DOUBLE,
  min_price  DOUBLE,
  max_price  DOUBLE,
  min_ts     TIMESTAMP,
  max_ts     TIMESTAMP,
  updated_at TIMESTAMP
) USING DELTA
""")

# Siembra (id=1) para simplificar la lógica del MERGE incremental en 03
spark.sql(f"""
MERGE INTO {T_RUNNING} t
USING (SELECT 1 AS id) s
ON t.id = s.id
WHEN NOT MATCHED THEN
  INSERT (id, row_count, sum_price, min_price, max_price, min_ts, max_ts, updated_at)
  VALUES (1, 0, 0.0, CAST(NULL AS DOUBLE), CAST(NULL AS DOUBLE),
          CAST(NULL AS TIMESTAMP), CAST(NULL AS TIMESTAMP), current_timestamp())
""")
print("[INFO] Tablas Oro listas y running_stats inicializado (id=1).")
